## --- Day 4: Giant Squid ---

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:
```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
 ```
 After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
Finally, 24 is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [91]:
class BingoBoard:

  def __init__(self, number_strings):
    self._numbers = [int(i) for i in number_strings]
    self._marked = 0
    self._win_lines = [32505856, 1015808, 31744, 992, 31, 17318416, 8659208,
                       4329604, 2164802, 1082401]
    self._bingo = False

  def check_if_bingo(self):    
    self._bingo = any([win_line & self._marked == win_line for win_line in self._win_lines])

    return self._bingo

  def call_and_check_if_bingo(self, called_number):
    # Stop marking/checking once Bingo is achieved
    if self._bingo:
      return False

    if called_number in self._numbers:
      i = self._numbers.index(called_number)
      self._marked = self._marked | (1 << (24-i))
    
    return self.check_if_bingo()

  def get_unmarked_numbers(self):
    marked_nums = set()
    board_nums = self._numbers.copy()
    marked = self._marked

    while marked > 0:
      if marked & 1 == 1:
        marked_nums.add(board_nums.pop())
      else:
        board_nums.pop()
      marked = marked >> 1

    return set(self._numbers).difference(marked_nums)

  def sum_unmarked_numbers(self):
    return sum(self.get_unmarked_numbers())


In [92]:
class BingoPlayer:
  def __init__(self, filename):
    self._all_numbers, self._boards = self.read_input(filename)
    self.winning_board = None
    self._last_called = None

  def play(self):
    while len(self._all_numbers) > 0 and not self.winning_board:
      this_number = self._all_numbers.pop(0)
      self._last_called = this_number
      for board in self._boards:
        if board.call_and_check_if_bingo(this_number):
          self.winning_board = board
          break

  def get_winning_score(self):
    if not self.winning_board:
      return None

    sum_unmarked = self.winning_board.sum_unmarked_numbers()

    return sum_unmarked * self._last_called

  def read_input(self, filename):
    with open(filename) as f:
      # First line is the number sequence, followed by a blank line
      called_numbers = f.readline().rstrip().split(',')
      called_numbers = [int(i) for i in called_numbers]
      _ = f.readline()
      # subsequent lines are boards
      boards = []
      board = []

      for line in f.readlines():
        if line != "\n":
          board += (line.split())
        else:
          boards.append(BingoBoard(board))
          board = []
      boards.append(BingoBoard(board))

    return (called_numbers, boards)

In [93]:
example1_player = BingoPlayer("./inputs/Day4ex.txt")
example1_solution = 4512

example1_player.play()
assert example1_solution == example1_player.get_winning_score()

In [94]:
player1 = BingoPlayer("./inputs/Day4.txt")
player1.play()
player1.get_winning_score()

35711

### --- Part Two ---

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

In [95]:
class BingoLoser(BingoPlayer):
  def __init__(self, filename):
    BingoPlayer.__init__(self, filename)
    self._boards_won = []
    self._called_numbers = []

  def play(self):
    while len(self._boards_won) < len(self._boards) and len(self._all_numbers) > 0:
      this_number = self._all_numbers.pop(0)
      self._called_numbers.append(this_number)
      for board in self._boards:
        if board.call_and_check_if_bingo(this_number):
          self._boards_won.append(board)
  
  def get_losing_score(self):
    if len(self._boards_won) == 0:
      return None

    losing_board = self._boards_won[-1]
    last_called = self._called_numbers[-1]

    return losing_board.sum_unmarked_numbers() * last_called

In [96]:
example2_loser = BingoLoser("./inputs/Day4ex.txt")
example2_solution = 1924

example2_loser.play()

assert example2_solution == example2_loser.get_losing_score()

In [97]:
part2_loser = BingoLoser("./inputs/Day4.txt")
part2_loser.play()
print(part2_loser.get_losing_score())

5586
